### Ingest drivers.json file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import lit

name_schema = StructType(
    fields=[
        StructField("forename", StringType(), True),
        StructField("surname", StringType(), True),
    ]
)

drivers_schema = StructType(
    fields=[
        StructField("driverId", IntegerType(), False),
        StructField("driverRef", StringType(), True),
        StructField("number", IntegerType(), True),
        StructField("code", StringType(), True),
        StructField("name", name_schema),
        StructField("dob", DateType(), True),
        StructField("nationality", StringType(), True),
        StructField("url", StringType(), True),
    ]
)

# Read from Landing Zone
drivers_df = spark.read.schema(drivers_schema).json(
    f"{landing_folder_path}/{v_file_date}/drivers.json"
)

##### Step 2 - Add ingestion meta data

In [ ]:
drivers_final_df = (
    add_ingestion_date(drivers_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write to output to bronze container

In [ ]:
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.drivers"
)